In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')
%load_ext google.colab.data_table

Authenticated


https://pythex.org/

In [ ]:
%%bigquery --project august-bucksaw-330517
select regexp_extract_all('i am walking my dog', r'\w+') as words

,words
0,"[i, am, walking, my, dog]"


In [ ]:
%%bigquery --project august-bucksaw-330517
select split('i am walking my dog', ' ') as words

,words
0,"[i, am, walking, my, dog]"


In [ ]:
%%bigquery --project august-bucksaw-330517
select title, id, split(title, ' ') words 
from `jrjames83-1171.sampledata.top_questions`
limit 2

,title,id,words
0,How to write unicode cross symbol in Java?,16616162,"[How, to, write, unicode, cross, symbol, in, J..."
1,"Windows batch file, wait for command to finish?",40193625,"[Windows, batch, file,, wait, for, command, to..."


In [ ]:
%%bigquery --project august-bucksaw-330517
with base_table as (select tag, title, id, split(title, ' ') words 
from `jrjames83-1171.sampledata.top_questions`)

select distinct tag, title, word 
from base_table, unnest(words) word
order by 2
limit 8

,tag,title,word
0,git-push,! [rejected] master -> master (fetch first),master
1,git-push,! [rejected] master -> master (fetch first),first)
2,git-push,! [rejected] master -> master (fetch first),->
3,git-push,! [rejected] master -> master (fetch first),[rejected]
4,git-push,! [rejected] master -> master (fetch first),(fetch
5,git-push,! [rejected] master -> master (fetch first),!
6,fast-forward,! [rejected] master -> master (non-fast-forwar...,->
7,fast-forward,! [rejected] master -> master (non-fast-forwar...,new


In [ ]:
%%bigquery --project august-bucksaw-330517
with base_table as (select tag, title, id, split(title, ' ') words 
from `jrjames83-1171.sampledata.top_questions`),

words_table as (select distinct tag, title, lower(trim(word)) word
from base_table, unnest(words) word)

select * from (
    select tag, word, count(*) frequency,
    row_number() over(partition by tag order by count(*) desc) tag_word_rank
    from words_table
    where word not in ('a', 'in', 'and', 'or', 'the', 'it', 'to', 'for', 'what', 'that', 'with', 'if', 'i') 
    and lower(tag) = 'python'
    group by 1,2
    order by 1,3 desc  
) wt where wt.tag_word_rank < 4
order by 1,4
limit 10

,tag,word,frequency,tag_word_rank
0,python,how,13,1
1,python,of,7,2
2,python,do,7,3


https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-preprocessing-functions#mlngrams

In [ ]:
%%bigquery --project august-bucksaw-330517
SELECT
  ML.NGRAMS(['a', 'b', 'c'], [2,3], '#') AS output

,output
0,"[a#b, a#b#c, b#c]"


In [ ]:
# 'i am walking my dog' / ['i'], ['am'], ['walking'], ['my'], ['dog']
# unigram: ['the', 'dog']
# bigrams: ['the dog', 'dog ran', 'ran quickly']
# trigram: ['the dog ran', 'dog ran quickly'
# NLP

%%bigquery --project august-bucksaw-330517
select ml.ngrams(['the', 'dog', 'ran', 'quickly'], [2,2]) bigrams

,bigrams
0,"[the dog, dog ran, ran quickly]"


In [ ]:
%%bigquery --project august-bucksaw-330517
select ml.ngrams(['the', 'dog', 'ran', 'quickly'], [1,2]) unigrams_bigrams

,unigrams_bigrams
0,"[the, the dog, dog, dog ran, ran, ran quickly,..."


In [ ]:
%%bigquery --project august-bucksaw-330517
with base_table as (select tag, title, id, ml.ngrams(split(title, ' '), [1,3]) words 
from `jrjames83-1171.sampledata.top_questions`),

words_table as (select distinct tag, title, lower(trim(word)) word
from base_table, unnest(words) word)

select * from (
    select tag, word, count(*) frequency,
    row_number() over(partition by tag order by count(*) desc) tag_word_rank
    from words_table
    where word not in ('a', 'in', 'and', 'or', 'the', 'it', 'to', 'for', 'what', 'that', 'with', 'if', 'i') 
    and lower(tag) = 'python'
    group by 1,2
    order by 1,3 desc  
) wt where wt.tag_word_rank < 4
order by 1,4
limit 10

,tag,word,frequency,tag_word_rank
0,python,how,13,1
1,python,how do,7,2
2,python,do,7,3


In [ ]:
%%bigquery --project august-bucksaw-330517
with base_table as (select tag, title, id, ml.ngrams(split(title, ' '), [1,3]) words 
from `jrjames83-1171.sampledata.top_questions`),

words_table as (select distinct tag, title, lower(trim(word)) word
from base_table, unnest(words) word),

ngrams_table as (select * from (
    select tag, word, count(*) frequency,
    row_number() over(partition by tag order by count(*) desc) tag_word_rank
    from words_table
    where word not in ('a', 'in', 'and', 'or', 'the', 'it', 'to', 'for', 'what', 'that', 'with', 'if', 'i') 
    and lower(tag) = 'python'
    group by 1,2
    order by 1,3 desc  
) wt where wt.tag_word_rank < 20
order by 1,4)

select *
from ngrams_table 
where array_length(split(word, ' ')) > 2
limit 10

,tag,word,frequency,tag_word_rank
0,python,how do i,6,5


In [ ]:
%%bigquery --project august-bucksaw-330517
WITH ngrams AS (
-- the ngrams generated from our documents
  SELECT id, title, ML.NGRAMS(SPLIT(title, ' ' ), [1,10]) as doc
  FROM `bigquery-samples.reddit.full`
)
-- we flatten the ngrams into a table, and JOIN to our names
SELECT distinct id, word, title
FROM ngrams, UNNEST(doc) as word
JOIN `bigquery-public-data.usa_names.usa_1910_2013` names ON names.name = word
order by 3
limit 10

,id,word,title
0,1283r4,Robert,\t\t\nUS Answer a Short Survey about where you...
1,1283r4,Griffin,\t\t\nUS Answer a Short Survey about where you...
2,16uuyo,Boston,\t\tTHE POST GAME THREAD: Chicago Bulls 100 - ...
3,shnny,Will,\t\nAmazon Will Lower ebook Prices
4,s4tw6,Bay,\t\nDutch Pirate Party Will Protect The Pirate...
5,s4tw6,Will,\t\nDutch Pirate Party Will Protect The Pirate...
6,pd6ez,Will,\t\nHolland Will Ban Torrent Trackers. In case...
7,pd6ez,In,\t\nHolland Will Ban Torrent Trackers. In case...
8,p368g,Will,\t\nMegaUpload Users Will Sue American Government
9,p368g,Sue,\t\nMegaUpload Users Will Sue American Government
